## Few-shots learning prompting

In [1]:
import warnings
import os

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

Load the environment

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


In [3]:
# create the OpenAI client, which can be used transparently with Anyscale Endpoints too

from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [4]:
# Utility function to send and fetch response

def get_commpletion(clnt: object, model: str, system_content: str, user_content:str) -> str:
    chat_completion = clnt.chat.completions.create(
        model=model,
    messages=[{"role": "system", "content": system_content},
              {"role": "user", "content": user_content}],
    temperature = 0.8)

    response = chat_completion.choices[0].message.content
    return response

## Recipe Generation
An example use case for few-shot learning is to provide couple of examples of recipies that the model can learn from and generalize. Using 
what it has learned as couple of in-context examples, it can generate a response for its subsequent prompt.

In [7]:
system_content = """You are supreme repository of knowledge, including all 
exotic culnary dishes across the globe."""

few_shot_recipe_prompt = """Recipe for simple cheese omlette. 
Ingridients: 
1. 3 large eggs
2. 1/4 cup shredded cheddar cheese
3. Salt and pepper to taste
4. 1 tablespoon butter
Directions:
1. Crack the eggs into a bowl, add salt, and pepper, then beat them until well mixed.
2. Heat the butter in a non-stick pan over medium heat until it melts and coats the pan.
3. Pour the beaten eggs into the pan, swirling them to cover the bottom evenly.
4. Sprinkle the shredded cheese over one half of the eggs.
5. Once the edges are set, fold the omelet in half with a spatula, covering the cheese. Cook for another minute until the cheese melts. Enjoy your delicious cheese omelet!

Generate a recipe for choclate pancakes:
Ingridients:
Directions:
"""

In [8]:
response = get_commpletion(client, MODEL, system_content, few_shot_recipe_prompt)
print(f"{response}\n")

Recipe for Chocolate Pancakes

Ingredients:
1. 1 cup all-purpose flour
2. 2 tablespoons cocoa powder (unsweetened)
3. 2 tablespoons sugar
4. 1 tablespoon baking powder
5. 1/2 teaspoon salt
6. 1 large egg
7. 1 cup milk
8. 2 tablespoons unsalted butter (melted)
9. 1 teaspoon vanilla extract
10. 1/2 cup chocolate chips (plus extra for garnish)
11. Butter or oil for cooking
12. Maple syrup or chocolate syrup for serving (optional)
13. Whipped cream for serving (optional)
14. Fresh berries for serving (optional)

Directions:
1. In a large mixing bowl, sift together the flour, cocoa powder, sugar, baking powder, and salt. Stir to combine the ingredients evenly.

2. In a separate bowl, beat the egg and then mix in the milk, melted butter, and vanilla extract until well combined.

3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix; a few lumps are okay. Gently fold in the chocolate chips.

4. Heat a non-stick pan or griddle over medium 

### Code Generation and Scripting

Using few-shot learning, we can provide in-context examples of how to write a script that the model can generalize and generate any script 
in the language its trained on.


In [21]:
system_content = """You are supreme repository of knowledge, and assistant
code Co-pilot for developer to assist them in generating sample code, inclding
in langauges such as Python, JavaScript, Java, C, C++, and shell script.
"""

few_shot_code_gen_prompt = """
Python code to compute Fibonacci series:

Problem: Compute Fibonnacci series
Language: Python
Code: 
def fibonacci(n):
    fib_series = [0, 1]
    while fib_series[-1] + fib_series[-2] <= n:
        fib_series.append(fib_series[-1] + fib_series[-2])
    return fib_series

# Example: Compute Fibonacci series up to 50
result = fibonacci(50)
print(result)

Write a JavaScript to read HTML contents from a Web URL. Follow
the format and labels provide below:
Problem: 
Language:
Code:
"""

In [20]:
response = get_commpletion(client, MODEL, system_content, few_shot_code_gen_prompt)
print(f"{response}\n")

Problem: Read HTML contents from a Web URL
Language: JavaScript
Code:
```javascript
const https = require('https');

function fetchHTMLContent(url) {
  https.get(url, (res) => {
    let data = '';

    // A chunk of data has been received.
    res.on('data', (chunk) => {
      data += chunk;
    });

    // The whole response has been received. Print out the result.
    res.on('end', () => {
      console.log(data);
    });

  }).on("error", (err) => {
    console.log("Error: " + err.message);
  });
}

// Example usage: Fetch HTML content from a specified URL
const url = 'https://www.example.com';
fetchHTMLContent(url);
```

Note: The above code snippet uses Node.js's built-in `https` module for making HTTP GET requests. If the content is served over HTTP, you should use the `http` module instead. Additionally, this code does not handle HTTPS certifications or redirections, which might be necessary for some websites. For a more robust solution, you might consider using a Node.js librar